In [ ]:
import deepxde as dde
import numpy as np
from itertools import combinations_with_replacement
from deepxde.backend import pytorch as torch

Рассмотрим 10 аналитически заданных функций $u(x,y; a,b) = \sin{a x}\cos{b y}$, где $a, b \in 1,2,3,4$ и $a \le b$. 
Тогда
$$-\Delta u(x,y) = (a^2 + b^2) u(x,y) = f(x,y) \Rightarrow f(x,y)=(a^2+b^2)\sin{ax}\cos{by},$$
$$u(0,y)=u(\pi, y)=0,\quad u(x, 0)=\sin{ax},\quad u(x,\pi)=(-1)^{b}\sin{ax}.$$

In [ ]:
examples_cfg = list(combinations_with_replacement((1, 2, 3, 4), 2))

assert len(examples_cfg) == 10

Реализуем функцию, которая решает задачу Дирихле на произвольном множестве $\Omega=geom$ с заданными $f(x,y)$ и $g(x,y)$:
$$
\begin{cases}
    -\Delta u = f,\quad (x,y) \in \Omega,\\
    u = g,\quad (x, y) \in \partial \Omega
\end{cases}
$$

In [ ]:
net = dde.nn.FNN([2, 50, 50, 50, 50, 1], "tanh", "Glorot uniform")

def solve_poisson(geom, f, g, n_iterations=1_000, solution=None, ckpt_path="model.ckpt", movie_path="movie"):
    def pde(x, y):
        du_xx = dde.grad.hessian(y, x, i=0, j=0)
        du_yy = dde.grad.hessian(y, x, i=1, j=1)
        return -du_xx - du_yy - f(x)

    def boundary(_, on_boundary):
        return on_boundary
    bc = dde.icbc.DirichletBC(geom, g, boundary)
    data = dde.data.PDE(geom, pde, bc, solution=solution, num_domain=1_000, num_boundary=500, num_test=1_000)

    model = dde.Model(data, net)
    model.compile("adam", lr=1e-3, metrics=["l2 relative error"])

    checkpointer = dde.callbacks.ModelCheckpoint(
        ckpt_path, verbose=1, save_better_only=True
    )
    # movie = dde.callbacks.MovieDumper(
    #     movie_path, [0., np.pi/2], [np.pi, np.pi/2], period=n_iterations//100, y_reference=solution
    # )
    losshistory, _ = model.train(iterations=n_iterations, callbacks=[checkpointer])#, movie])

    xy = geom.random_points(n=1_000)
    u_true, u_pred = solution(xy), model.predict(xy)
    residual = model.predict(xy, operator=pde)
    print(
        f"Solution L2 absolute error: {np.linalg.norm(u_true - u_pred, ord=2)}",
        f"Solution L2 relative error: {np.linalg.norm(u_true - u_pred, ord=2) / np.linalg.norm(u_true, ord=2)}",
        f"Solution C absolute error: {np.max(np.abs(u_true - u_pred))}",
        f"Solution C relative error: {np.max(np.abs(u_true - u_pred)) / np.max(np.abs(u_true))}",
        f"Residual L2 error: {np.linalg.norm(residual, ord='fro')}",
        f"Residual C error: {np.max(np.abs(residual))}",
        sep="\n"
    )
    return model, losshistory

In [ ]:
geom = dde.geometry.Rectangle([0., 0.], [np.pi, np.pi])
n_iterations = 1_000

results = {}
for a, b in examples_cfg:
    def f(x):
        return (a**2 + b**2) * torch.sin(a * x[:, 0]) * torch.cos(b * x[:, 1])

    def g(x):
        boundary = np.zeros_like(x[:, 0])
        on_boundary = dde.utils.isclose(x[:, 1], 0.) | dde.utils.isclose(x[:, 1], np.pi)
        boundary[on_boundary] = np.sin(a * x[:, 0])[on_boundary]
        return boundary

    def solution(x):
        return np.sin(a * x[:, 0]) * np.cos(b * x[:, 1])

    print(f"Training model for u(x,y)=sin({a}x)*cos({b}y)")
    model, losshistory = solve_poisson(geom, f, g, n_iterations=n_iterations, solution=solution)
    results[(a, b)] = {
        "model": model,
        "losshistory": losshistory
    }